In [185]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np 

In [186]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [187]:
docs.count_documents(filter={})

0

In [188]:
docs.drop()

## Here is how i want the scrape to work

    """LOS = [List of Strains]
        for s in LOS:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        for i in range(1,100):
            d = get_reviews(url+str(i))
            st, rev = parse_docs(d)
            if len(st) > 0:
               append.(add to your dictionary)
            else:
                break"""

In [13]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [14]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [15]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [16]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
    return star_num

In [193]:
def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
         LOS.append(str(s.attrs['href'])[1:])
    return LOS

In [194]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_rate = []
    user_name = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        strain_text.append(r.text)
    tags = soup.select("div.div.stars")
    star = []
    for t in tags:
        star.append(t.attrs['style'])
    star_rate.append(star_int_conv(star))
    users = soup.find_all('div', class_='strain-review__title')
    for u in users:
        temp = u.find('h2')
        user_name.append(temp.text)
    return star_rate, strain_text, user_name

In [195]:
#Test works better, is updated!
def scraper_dummy1(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            print(users)
            if len(star[0]) == 0:
                break

In [196]:
def scraper_dummy_test(los):
    los_urls = []
    rev_urls = []
    user_list_of_lists = []
    star_list_of_lists = []
    reviews_list_o_lists = []
    #grab URL landing pages for reviews on each strain.
    for s in los:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        strain_star_list = []
        strain_user_list = []
        strain_rev_list = []
        #get all reviews, stars, and users for each strain in list of strains
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            #return stars, strains, reviews, in one list, not list of lists for each page scraped. 
            #star_unlisted = 
            strain_star_list.extend([x for x in star[0]])
            strain_user_list.extend(users)
            strain_rev_list.extend(reviews)
            if len(star[0]) == 0:
                break
        #these are long single lists, not list of lists
        star_list_of_lists.append(strain_star_list)
        user_list_of_lists.append(strain_user_list)
        reviews_list_o_lists.append(strain_rev_list)
        #make into dictionaries 
        for s in los:
            star_ds = []
            rev_ds = []
            for i in range(len(star_list_of_lists)):
                user_revs =  dict(list(zip(user_list_of_lists[i], reviews_list_o_lists[i])))
                rev_ds.append(user_revs)
                user_stars = dict(list(zip(user_list_of_lists[i], star_list_of_lists[i])))
                star_ds.append(user_stars)
    #create Pandas DF of each dictionary as well.
    star_dfs = []
    rev_dfs = []
    for s in star_ds:
        star_dfs.append(pd.DataFrame.from_dict(s, orient='index')) 
    for r in rev_ds: 
        rev_dfs.append(pd.DataFrame.from_dict(r, orient ='index'))
    #create large dataframe
    star_df = pd.concat(star_dfs, axis=1, sort=True)
    rev_df = pd.concat(rev_dfs, axis=1, sort=True)
    rev_df.columns = los
    star_df.columns = los
    return star_df, rev_df

In [197]:
test_strains = list_o_strains('https://www.leafly.com/explore')

In [198]:
test_strains[6]

'hybrid/original-glue'

In [199]:
stars, reviews = scraper_dummy_test(test_strains[6:8])

In [200]:
stars

,hybrid/original-glue,hybrid/white-widow
13Gawd,5.0,NaN
152gjeng,NaN,3.0
1taste,NaN,3.0
206SEA,5.0,NaN
419plus1,NaN,4.0
420life,5.0,NaN
420reyes420,5.0,NaN
561.welly,5.0,NaN
8CoreXeon,NaN,4.0
8characters,5.0,NaN
